In [1]:
import os
import sys
import math
import psutil
os.environ["_CCTRACE_"]="ON"
import cloudComPy as cc
from gendata import getSampleCloud, dataDir
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
print("Main : Libs loaded")

Main : Libs loaded


In [2]:
# TODO Load the data (using laspy -> cloudComPy)
# TODO optional convert into pandas dataframe (pandas limit to 100mb?)
# TODO Test functions:
# - getPointCloud
# - getSphere
# 
# TODO Slice the data
# TODO Subtract the data, need to ease the surface with gaussian fucntion, search the net for examples
# TODO Visualize the data

In [3]:
path1 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_7_42.LiData"
path2 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.las"
path3 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.LiData"
path4 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.las"
path5 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.LiData"
assert os.path.isfile(path1)
assert os.path.isfile(path2)
assert os.path.isfile(path3)
assert os.path.isfile(path4)
assert os.path.isfile(path5)
# with open(file=path, mode="r") as cloudFile:
#     print("Main : File opened")

In [4]:
#* os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

#* import cloudComPy as cc                                                # import the CloudComPy module

#* cloud = cc.loadPointCloud("myCloud.xyz")                               # read a point cloud from a file
#* print("cloud name: %s"%cloud.getName())

#* res=cc.computeCurvature(cc.CurvatureType.GAUSSIAN_CURV, 0.05, [cloud]) # compute curvature as a scalar field
#* nsf = cloud.getNumberOfScalarFields()
#* sfCurv=cloud.getScalarField(nsf-1)
#* cloud.setCurrentOutScalarField(nsf-1)
#* filteredCloud=cc.filterBySFValue(0.01, sfCurv.getMax(), cloud)         # keep only the points above a given curvature

#* ok = filteredCloud.exportCoordToSF(False, False, True)                 # Z coordinate as a scalar Field
#* nsf = cloud.getNumberOfScalarFields()
#* sf1=filteredCloud.getScalarField(nsf-1)
#* mean, var = sf1.computeMeanAndVariance()

#* # using Numpy...

#* coordinates = filteredCloud.toNpArrayCopy()                            # coordinates as a numpy array
#* x=coordinates[:,0]                                                     # x column
#* y=coordinates[:,1]
#* z=coordinates[:,2]

#* f=(2*x-y)*(x+3*y)                                                      # elementwise operation on arrays

#* asf1=sf1.toNpArray()                                                   # scalar field as a numpy array
#* sf1.fromNpArrayCopy(f)                                                 # replace scalar field values by a numpy array

#* res=cc.SavePointCloud(filteredCloud,"myModifiedCloud.bin")             #save the point cloud to a file

In [6]:
# Using path2 as input (This is .las data file)
cc.initCC()
with open(path2, "rb") as cloudFileHeader:
    if type(cloudFileHeader) != None:
        cloudFile = laspy.read(cloudFileHeader)
        try:
            print("Cloud loaded")
            print(f"cloud name:{cloudFile.header}")
        except:
            print(f"cloud name:<unknown>")

cloud name:<LasHeader(1.4, <PointFormat(7, 0 bytes of extra dims)>)>
